In [1]:
import boto3

In [ ]:
ec2 = boto3.resource('ec2', region_name='ap-southeast-2')

instances = ec2.create_instances(
    ImageId="ami-0d37dd644530396ea",
    MinCount=1,
    MaxCount=1,
    InstanceType="t2.medium",
    KeyName="w5"
)

In [5]:
print(instances)

NameError: name 'instances' is not defined

In [66]:
client = boto3.client('ec2', region_name='ap-southeast-2')

# response = client.terminate_instances(
#     InstanceIds=[
#         'i-0fe5c971615905c75',
#     ],
# )

response = client.start_instances(
    InstanceIds=[
        'i-03f99e382069a2c98',
    ],
)

In [61]:
client = boto3.client('ec2', region_name='ap-southeast-2')

response = client.stop_instances(
    InstanceIds=[
        'i-03f99e382069a2c98',
    ],
)

In [6]:
def get_running_instances():
    ec2_client = boto3.client('ec2', region_name='ap-southeast-2')
    reservations = ec2_client.describe_instances(Filters=[
        {
            "Name": "instance-state-name",
            "Values": ["running"],
        }
    ]).get("Reservations")
    
    if not reservations:
        print("No instance is running now")
    else:
        for reservation in reservations:
            for instance in reservation["Instances"]:
                instance_id = instance["InstanceId"]
                instance_type = instance["InstanceType"]
                public_ip = instance["PublicIpAddress"]
                private_ip = instance["PrivateIpAddress"]
                print(f"{instance_id}, {instance_type}, {public_ip}, {private_ip}")

In [7]:
def get_public_ip(instance_id):
    ec2_client = boto3.client('ec2', region_name='ap-southeast-2')
    reservations = ec2_client.describe_instances(InstanceIds=[instance_id]).get("Reservations")

    for reservation in reservations:
        for instance in reservation['Instances']:
            print(instance.get("PublicIpAddress"))

In [76]:
## Run command against your linux VM
def runRemoteShellCommands (InstanceId):
    ssm_client = boto3.client('ssm', region_name="ap-southeast-2")
#     commands = ['sudo systemctl status amazon-ssm-agent']
    response = ssm_client.send_command(
        InstanceIds=[InstanceId],
#        DocumentName="AWS-RunShellScript",
        DocumentName="AWS-RunRemoteScript",
#        Parameters={'commands':commands}
        Parameters= {
            "sourceType": [
                "GitHub"
            ],
            "commandLine": [
                "client.py"
            ],
            "sourceInfo": [
                "{\"owner\":\"YiYang-Clotho\", \"repository\":\"FedAvg-simulation\", \"path\": \"\"}"
            ]
        }
      )
    command_id = response['Command']['CommandId']
    output = ssm_client.get_command_invocation(
        CommandId=command_id, 
        InstanceId=InstanceId
    )
    while output['Status'] == "InProgress":  
        output = ssm_client.get_command_invocation(CommandId=command_id, InstanceId=InstanceId) 
#     print(output['StandardOutputContent'])
    print(output)

In [77]:
get_running_instances()

i-03f99e382069a2c98, t2.medium, 3.26.179.170, 172.31.24.39


In [78]:
get_public_ip('i-03f99e382069a2c98')

3.26.179.170


In [79]:
commands = ['{"sourceType":["GitHub"],"sourceInfo":["{\"owner\":\"YiYang-Clotho\", \"repository\":\"FedAvg-simulation\", \"path\": \""}"],"commandLine":["client.py"]}']
runRemoteShellCommands("i-03f99e382069a2c98")

{'CommandId': '09a9824d-862e-4495-91ad-b3545ab50b9b', 'InstanceId': 'i-03f99e382069a2c98', 'Comment': '', 'DocumentName': 'AWS-RunRemoteScript', 'DocumentVersion': '$DEFAULT', 'ResponseCode': 0, 'ExecutionEndDateTime': '', 'Status': 'Failed', 'StatusDetails': 'Failed', 'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '', 'CloudWatchOutputEnabled': False}, 'ResponseMetadata': {'RequestId': 'a08e4e8a-d993-461c-9ac1-c9fad56b7d73', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Mon, 22 May 2023 10:33:08 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '339', 'connection': 'keep-alive', 'x-amzn-requestid': 'a08e4e8a-d993-461c-9ac1-c9fad56b7d73'}, 'RetryAttempts': 0}}
